In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import pandas as pd
import yaml
from itertools import chain, product, combinations
import torch

from time import time as tt

sys.path.append("../../../")
from gnn4itk_cf.stages.data_reading.models.trackml_utils import *

from gnn4itk_cf.stages.data_reading.data_reading_stage import EventReader
from gnn4itk_cf.stages.data_reading.models.trackml_reader import TrackMLReader

from gnn4itk_cf.stages.graph_construction.models.metric_learning import MetricLearning
from gnn4itk_cf.stages.edge_classifier.models.filter import Filter
from gnn4itk_cf.stages.edge_classifier import InteractionGNN

from gnn4itk_cf.stages.graph_construction.utils import handle_weighting
from gnn4itk_cf.stages.graph_construction.models.utils import graph_intersection, build_edges
from gnn4itk_cf.stages.graph_construction.utils import *

/users/PLS0129/ysu0053/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/users/PLS0129/ysu0053/.conda/envs/torch/lib/python3.10/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/users/PLS0129/ysu0053/.conda/envs/torch/lib/python3.10/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
#config_file = "trackml_reader_config.yaml"
#config = yaml.load(open(config_file, "r"), Loader=yaml.FullLoader)

In [3]:
input_dir = "/fs/scratch/PLS0144/alina/TrackML_Results/Example_3/feature_store/trainset"
sample = torch.load(os.path.join(input_dir, "event000001001-graph.pyg"), map_location="cpu")
sample

Data(hit_id=[93680], x=[93680], y=[93680], z=[93680], r=[93680], phi=[93680], eta=[93680], region=[93680], module_index=[93680], weight=[93680], cell_count=[93680], cell_val=[93680], leta=[93680], lphi=[93680], lx=[93680], ly=[93680], lz=[93680], geta=[93680], gphi=[93680], track_edges=[2, 68512], particle_id=[68512], pt=[68512], radius=[68512], nhits=[68512], config=[1], event_id='000001001')

In [4]:
with open("../examples/Example_3/metric_learning_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_ML = MetricLearning(config)
with open("../examples/Example_3/filter_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_filter = Filter(config)
with open("../examples/Example_3/gnn_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_gnn = InteractionGNN(config)

In [5]:
model_ML.setup(stage="predict")
dataloaders = model_ML.predict_dataloader()
#dataloaders[2].dataset.__dict__
model_filter.setup('predict')
model_gnn.setup('predict')

Loaded 80 training events, 10 validation events and 10 testing events


/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [pid] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [n_hits] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [primary] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [pdg_id] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data")
/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/utils/loading_utils.py:65: UserWarning: OPTIONAL feature [ghost] not found in data
  warnings.warn(f"OPTIONAL feature [{feature}] not found in data"

Defining figures of merit


/users/PLS0129/ysu0053/CommonFramework/gnn4itk_cf/stages/edge_classifier/edge_classifier_stage.py:105: UserWarning: Failed to define figures of merit, due to logger unavailable
  warnings.warn("Failed to define figures of merit, due to logger unavailable")


Defining figures of merit


In [6]:
model_ML = model_ML.to("cuda")
model_filter = model_filter.to("cuda")
model_gnn = model_gnn.to("cuda")
for batch in dataloaders[2]:
    batch = batch.to("cuda")
    with torch.no_grad():
        embedding = model_ML.apply_embedding(batch)
     
    batch.edge_index = build_edges(
        query=embedding, database=embedding, indices=None, r_max=0.1, k_max=10, backend="FRNN"
    )
    R = batch.r**2 + batch.z**2
    flip_edge_mask = R[batch.edge_index[0]] > R[batch.edge_index[1]]
    batch.edge_index[:, flip_edge_mask] = batch.edge_index[:, flip_edge_mask].flip(0)
    #out = model_filter(batch)   
    #preds = torch.sigmoid(out) > 0.5
    #update batch.edge_index
    out = model_gnn(batch)
    preds = torch.sigmoid(out) > 0.5
    sample=batch
    print(out)

tensor([-0.2744, -0.2815, -0.2762,  ..., -0.4961, -0.4754, -0.4470],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.2607, -0.2737, -0.3098,  ..., -0.6105, -0.5101, -0.5170],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.3479, -0.3113, -0.3837,  ..., -0.5017, -0.5604, -0.6177],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.3675, -0.3179, -0.2909,  ..., -0.3837, -0.3564, -0.3487],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.1912, -0.1819, -0.3027,  ..., -0.6042, -0.6206, -0.3956],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.3544, -0.2981, -0.3024,  ...,  0.0816, -0.3910, -0.7525],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.3228, -0.3546, -0.3224,  ..., -0.1380, -0.1942, -0.1899],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.2810, -0.2733, -0.3419,  ..., -0.4334, -0.4855, -0.5197],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.2644, -0.3559, -0.343

In [8]:
export_output = torch.onnx.dynamo_export(model_filter, sample)

AttributeError: module 'torch.onnx' has no attribute 'dynamo_export'